In [1]:
import pandas as pd
import os

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import unicodedata
import re

In [2]:
path = os.path.join(os.getcwd(), 'Data', 'input_search_DB.csv')
df = pd.read_csv(path, sep=',', encoding='utf-8')
df.head()

,order_id,order_customer_name,product_name,part_type_name,product_postpress_type_name
0,2200006,得意先_1,A 2023年3月号 定期演奏会,['本文1'],NaN
1,2107551,得意先_7,アーティストリスト2022年,"['本文1', '本文1', '本文2', '本文2', '表紙1', '表紙1']",NaN
2,2200898,得意先_8,ミュージアムリーフレット,['本文'],NaN
3,2202767,得意先_148,A小学校 2024学校案内パンフレット,"['本文1', '本文2', '表紙1', '表紙2']",NaN
4,2203087,得意先_14,A社統合報告書2022（英文）,"['本文1', '表紙1']",NaN


In [3]:
df = df[['order_id', 'order_customer_name', 'product_name', 'part_type_name']]
df.head()

,order_id,order_customer_name,product_name,part_type_name
0,2200006,得意先_1,A 2023年3月号 定期演奏会,['本文1']
1,2107551,得意先_7,アーティストリスト2022年,"['本文1', '本文1', '本文2', '本文2', '表紙1', '表紙1']"
2,2200898,得意先_8,ミュージアムリーフレット,['本文']
3,2202767,得意先_148,A小学校 2024学校案内パンフレット,"['本文1', '本文2', '表紙1', '表紙2']"
4,2203087,得意先_14,A社統合報告書2022（英文）,"['本文1', '表紙1']"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   order_id             98 non-null     int64 
 1   order_customer_name  98 non-null     object
 2   product_name         98 non-null     object
 3   part_type_name       98 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


In [5]:
def normalize(text):
    text = unicodedata.normalize("NFKC", str(text))
    text = re.sub(r'\s+', '', text)
    return text

In [8]:
def fuzzy_match_df(input_text, df, flag=False, top_n=10, scorer = 'ratio'):
    scorer_map = {
        'ratio': fuzz.ratio,
        'partial_ratio': fuzz.partial_ratio,
        'token_sort_ratio': fuzz.token_sort_ratio,
        'token_set_ratio': fuzz.token_set_ratio
    }

    if scorer not in scorer_map:
        raise ValueError(f"Invalid scorer '{scorer}'. Choose from: {list(scorer_map.keys())}")

    scoring_func = scorer_map[scorer]

    input_proc = normalize(input_text) if flag else input_text

    match_records = []

    for idx, row in df.iterrows():
        for col in df.columns:
            if col in ['order_id']:
                continue
            col_value = row[col]
            if pd.isna(col_value):
                continue
            
            current = normalize(col_value) if flag else col_value
            score = scoring_func(input_proc, current) if current else 0

            match_records.append({
                'order_id': row['order_id'],
                'order_customer_name': row['order_customer_name'],
                'product_name': row['product_name'],
                'part_type_name': row['part_type_name'],
                'matched_column': col,
                'match_score': score
            })

    result_df = pd.DataFrame(match_records)
    result_df = result_df.sort_values(by='match_score', ascending=False).head(top_n).reset_index(drop=True)
    return result_df

In [6]:
def fuzzy_match_column_df(input_text, df, column, flag=False, top_n=5, scorer='ratio'):
    scorer_map = {
        'ratio': fuzz.ratio,
        'partial_ratio': fuzz.partial_ratio,
        'token_sort_ratio': fuzz.token_sort_ratio,
        'token_set_ratio': fuzz.token_set_ratio
    }
    if scorer not in scorer_map:
        raise ValueError(f"Invalid scorer '{scorer}'. Choose from: {list(scorer_map.keys())}")
    
    scoring_func = scorer_map[scorer]
    input = normalize(input_text) if flag else input_text

    candidates = df[column].dropna().tolist()
    processed_candidates = [normalize(c) if flag else c for c in candidates]

    matches = process.extract(input, processed_candidates, limit=top_n, scorer=scoring_func)

    rs = []
    for match_text, score in matches:
        org_row = df[df[column].apply(lambda x: normalize(x) if flag else x) == match_text].iloc[0]
        row = org_row.to_dict()
        row['match_score'] = score
        rs.append(row)

    return pd.DataFrame(rs)

In [13]:
input = "ーセプル"
rs1 = fuzzy_match_df(input, df, flag=False)
rs1.head(10)

,order_id,order_customer_name,product_name,part_type_name,matched_column,match_score
0,2204467,得意先_179,I学園 学科ツール,"['本文1', '本文2', '表紙1']",product_name,31
1,2204548,得意先_134,3月メニュー,"['1台目', '2台目']",product_name,20
2,2203259,得意先_17,ブリスターパック台紙テストプリント,"['台紙', '台紙２']",product_name,19
3,2204327,得意先_158,Cメニュー お花見セット（型抜き）,"['本文1', '本文1', '本文1']",product_name,18
4,2204601,得意先_138,選挙啓発ポスター,['本文'],product_name,17
5,2204276,得意先_158,Aメニュー（春）,"['本文', '本文', '本文1', '本文2']",product_name,17
6,2204802,得意先_200,7週（2/14付） A社商品コンセプト,['PVT66W'],product_name,17
7,2204801,得意先_200,7週（2/14付） A社商品コンセプト,"['しらおい上質86.5', 'ユニフェイス14㎏']",product_name,17
8,2203843,得意先_27,2023 母の日用 ティーチケットセット,"['チケット', 'チケット・台紙', 'チケット・台紙', '台紙']",product_name,16
9,2204124,得意先_59,（仮）フォルダ\t\t,"['ペラ', 'ペラ', '冊子本文', '冊子表紙']",product_name,15


In [17]:
input = "ーセプル"
rs2 = fuzzy_match_df(input, df, flag=True, scorer = 'partial_ratio')
rs2.head(10)

,order_id,order_customer_name,product_name,part_type_name,matched_column,match_score
0,2204467,得意先_179,I学園 学科ツール,"['本文1', '本文2', '表紙1']",product_name,50
1,2204358,得意先_91,Aコンビニ 売場変更にともなう取消スケジュール冊子,"['1-2折', '3折']",product_name,50
2,2204802,得意先_200,7週（2/14付） A社商品コンセプト,['PVT66W'],product_name,50
3,2204801,得意先_200,7週（2/14付） A社商品コンセプト,"['しらおい上質86.5', 'ユニフェイス14㎏']",product_name,50
4,2203843,得意先_27,2023 母の日用 ティーチケットセット,"['チケット', 'チケット・台紙', 'チケット・台紙', '台紙']",product_name,25
5,2203259,得意先_17,ブリスターパック台紙テストプリント,"['台紙', '台紙２']",product_name,25
6,2204785,得意先_200,D 9週（2/28付） POP,"['Ne-tak', 'しらおい上質86.5kg', 'クリスパー0.25']",part_type_name,25
7,2204784,得意先_200,D 8週（2/21付） POP,"['Ne-tak', 'しらおい上質86.5kg', 'クリスパー0.25']",part_type_name,25
8,2204757,得意先_179,〇〇〇〇〇 必勝A1ポスター,['部品1'],product_name,25
9,2204728,得意先_169,"A社家具メーカー様 取扱説明書「グランサリー2モーター」【修正増刷】管理No,07-●●●●●●●",['表紙本文'],product_name,25


In [19]:
rs3 = fuzzy_match_column_df(input, df, 'product_name', flag=True, scorer='ratio')
rs3.head(5)

,order_id,order_customer_name,product_name,part_type_name,match_score
0,2204467,得意先_179,I学園 学科ツール,"['本文1', '本文2', '表紙1']",33
1,2204327,得意先_158,Cメニュー お花見セット（型抜き）,"['本文1', '本文1', '本文1']",21
2,2204124,得意先_59,（仮）フォルダ\t\t,"['ペラ', 'ペラ', '冊子本文', '冊子表紙']",20
3,2204548,得意先_134,3月メニュー,"['1台目', '2台目']",20
4,2203259,得意先_17,ブリスターパック台紙テストプリント,"['台紙', '台紙２']",19


In [12]:
rs4 = fuzzy_match_column_df(input, df, 'part_type_name', flag=True, scorer='ratio')
rs4.head(5)

,order_id,order_customer_name,product_name,part_type_name,match_score
0,2204644,得意先_148,T大学第一小学校 2024学校案内 ポスター＆チラシ,['ポスター&チラシ'],17
1,2204248,得意先_200,B POP,"['Ne-tak', 'しらおい上質86.5kg', 'クリスパー0.25']",5
2,2204248,得意先_200,B POP,"['Ne-tak', 'しらおい上質86.5kg', 'クリスパー0.25']",5
3,2204248,得意先_200,B POP,"['Ne-tak', 'しらおい上質86.5kg', 'クリスパー0.25']",5
4,2204248,得意先_200,B POP,"['Ne-tak', 'しらおい上質86.5kg', 'クリスパー0.25']",5
